# 🔬 Deep Learning: CNN, RNN y Transformers

Este notebook presenta una introducción teórica y práctica a tres de las arquitecturas más importantes en deep learning:

- **CNN (Convolutional Neural Networks)** – imágenes
- **RNN (Recurrent Neural Networks)** – series temporales y texto secuencial
- **Transformers** – procesamiento paralelo de secuencias, NLP moderno
- **GNN (Graph Neural Networkd)** - relaciones de nodos y aristas

Además, se incluye una introducción práctica a **TensorFlow y Keras**, las librerías más utilizadas para construir redes neuronales.


## Intro a Tensorflow y Keras

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print("TensorFlow version:", tf.__version__)

### ¿Qué es TensorFlow?

TensorFlow es una plataforma open-source desarrollada por Google para construir y entrenar modelos de machine learning y deep learning. Permite usar CPU, GPU o TPU.

### ¿Qué es Keras?

Keras es una API de alto nivel integrada en TensorFlow para construir modelos de forma más rápida, modular e intuitiva.


## CNN - Convolutional Neural Network

### ¿Qué es una CNN?

Las redes convolucionales están diseñadas para trabajar con datos que tienen estructura espacial, como imágenes. Aprenden **filtros** que extraen patrones como bordes, formas y texturas.

🧠 **Usos típicos**: Clasificación de imágenes, detección de objetos, reconocimiento facial.


### Ejemplo con el dataset de MNIST

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Cargar datos
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocesar
x_train = x_train[..., tf.newaxis] / 255.0
x_test = x_test[..., tf.newaxis] / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Modelo CNN simple
model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),  # Capa convolucional: 32 filtros 3x3
    layers.MaxPooling2D((2,2)),  # Reducción de dimensionalidad (stride 2)
    layers.Flatten(),            # Aplanar matriz 2D a vector 1D
    layers.Dense(64, activation='relu'),  # Capa densa totalmente conectada
    layers.Dense(10, activation='softmax')  # Salida con 10 clases y softmax (clasificación multiclase)
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

- Conv2D extrae patrones locales (bordes, formas).
- MaxPooling2D reduce resolución y ayuda a evitar overfitting.
- Flatten convierte los datos en formato vectorial para las capas densas.
- Dense realiza la clasificación como en una red neuronal tradicional.
- Softmax transforma los logits a probabilidades (suma = 1).

## RNN - Recurrent Neural Network

### ¿Qué es una RNN?

Las RNN están diseñadas para procesar datos secuenciales, como texto, audio o series temporales. Mantienen una "memoria" interna que ayuda a relacionar datos en diferentes pasos de la secuencia.

🧠 **Usos típicos**: Predicción de texto, traducción automática, análisis de sentimientos, predicción de series temporales.

### Ejemplo de generar texto con un RNN LSTM

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

corpus = [
    "I love deep learning",
    "Deep learning loves me",
    "I love TensorFlow",
    "I love AI"
]

# Tokenizar texto
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
X = pad_sequences(sequences, padding='post')

# Modelo LSTM simple
model = keras.Sequential([
    layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=8), # Codificación de palabras en vectores densos
    layers.LSTM(16),                                                       # Capa recurrente con 16 unidades
    layers.Dense(1, activation='sigmoid')                                  # Salida binaria (0 o 1)
])

model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

- Embedding convierte tokens a vectores densos aprendibles (ideal para texto).
- LSTM puede “recordar” secuencias largas mejor que una RNN simple.
- Dense + sigmoid sirve para clasificación binaria.

## Transformers

### ¿Qué es un Transformer?

Los Transformers procesan secuencias en paralelo (no paso a paso como las RNN) utilizando mecanismos de **atención** para asignar "peso" a las partes más relevantes de la secuencia.

🧠 **Usos típicos**: Chatbots, traducción automática, clasificación de texto, modelos como BERT y GPT.


### Ejemplo de un Transformer pequeño en Keras

In [ ]:
# Capa de atención simple (demo)
class SimpleSelfAttention(layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.q = layers.Dense(units)  # Capa para calcular queries
        self.k = layers.Dense(units)  # Capa para calcular keys
        self.v = layers.Dense(units)  # Capa para calcular values

    def call(self, x):
        q = self.q(x)
        k = self.k(x)
        v = self.v(x)
        attention = tf.nn.softmax(tf.matmul(q, k, transpose_b=True), axis=-1)  # Atención: similitud entre queries y keys
        return tf.matmul(attention, v)  # Pondera los values por atención

# Aplicación a una secuencia
inputs = layers.Input(shape=(10, 64))  # Secuencia de 10 pasos con 64 features
attention_output = SimpleSelfAttention(32)(inputs)
model = keras.Model(inputs, attention_output)
model.summary()

Este módulo calcula atención entre todos los elementos de la secuencia:

- Usa producto escalar entre query y key → qué tan relacionados están los tokens
- Luego multiplica por value → recoge la información relevante

## GNN - Graph Neural Networks

Las GNN permiten trabajar con datos que tienen estructura de grafo (nodos y conexiones), como redes sociales, moléculas, mapas, etc.

Cada nodo aprende representaciones vectoriales (embeddings) **considerando sus vecinos**.

🧠 **Usos típicos**:
- Recomendaciones
- Predicción molecular
- Análisis de relaciones (Knowledge Graphs)

Librerías comunes: PyTorch Geometric, DGL (Deep Graph Library)


### Ejemplo conceptual

In [ ]:
# PyTorch Geometric: modelo simple GCN
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


## Comparativa

| Arquitectura | Ideal para...                    | Ventajas                          | Desventajas                  |
|--------------|----------------------------------|-----------------------------------|------------------------------|
| CNN          | Imágenes, video, visión computarizada | Muy eficiente en 2D, pocos parámetros (patrones espaciales) | No sirve para secuencias     |
| RNN / LSTM   | Texto, audio, series temporales       | Recuerda contexto de pasos previos | Más lentas, difícil de entrenar |
| Transformers | Texto largo, NLP moderno             | Procesamiento paralelo, más potentes | Requiere mucho dato y cómputo |
| GNN         | Grafos, redes, moléculas | Aprende de estructura relacional | Requiere librerías específicas |


## (ANEXO) Transfer Learning & Fine-Tuning

El Transfer Learning consiste en **reutilizar un modelo preentrenado** (normalmente en un dataset grande como ImageNet) y adaptarlo a tu problema específico.

Hay dos enfoques:

1. **Feature Extraction**: Congelas el modelo base y solo entrenas una nueva capa de salida.
2. **Fine-Tuning**: Permites que algunas capas del modelo base sigan aprendiendo en tu dataset.


### Ejemplo de Transfer Learning con MobileNetV2

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input

# Cargar modelo preentrenado sin la capa final
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelamos todas las capas inicialmente
base_model.trainable = False

# Añadimos nuestra cabeza de clasificación
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
outputs = Dense(2, activation='softmax')(x)
model = Model(inputs, outputs)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

### Fine-Tuning: Descongelar y reentrenar algunas capas

In [ ]:
# Ahora permitimos entrenar las últimas capas del modelo base
base_model.trainable = True

# Podemos elegir cuántas capas descongelar
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Recompilar (cambiar learning rate)
model.compile(optimizer=keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Reentrenar con fine-tuning
history = model.fit(train_ds, validation_data=val_ds, epochs=5)

## (ANEXO 2) Fine Tuning en otro tipo de modelos

Fine-tuning (ajuste fino) significa tomar un modelo ya entrenado (o preentrenado) y ajustarlo ligeramente para un nuevo problema o conjunto de datos específico.

**¿Por qué hacer Fine Tuning?**
- Para ahorrar tiempo y cómputo, ya que reutilizas pesos aprendidos.
- Para mejorar performance, aprovechando el conocimiento previo.
- Ideal cuando tienes poco dato propio.

### Fine Tuning con Deep Learning

En resumen:

1. Cargas un modelo preentrenado (por ejemplo, una CNN de imagen entrenada en
ImageNet).
2. Le quitas o adaptas la última capa de salida a tu tarea.
3. Conge las capas iniciales (no se entrenan).
4. Entrenas solo la parte final.
5. Luego “descongelas” algunas capas y vuelves a entrenar todo con un learning rate más bajo.

### Fine-tuning en modelos de árboles (Random Forest, XGBoost)

Puedes ajustar hiperparámetros basándote en un modelo ya entrenado.

En XGBoost o LightGBM, puedes usar el parámetro xgb_model=model_previo para seguir entrenando.

In [ ]:
import xgboost as xgb

# Dataset inicial
model1 = xgb.train(params, dtrain, num_boost_round=50)

# Fine-tuning con más datos
model2 = xgb.train(params, dtrain_new, num_boost_round=20, xgb_model=model1)

### Fine-tuning en modelos lineales (Regresión, SVM)

No se hace "fine-tuning" como tal, pero puedes:

- Usar los coeficientes de un modelo anterior como punto de partida para otro modelo (en frameworks que lo permiten).
- Cambiar regularización (C en SVM, alpha en Ridge/Lasso) y volver a entrenar con datos nuevos.

### Fine-tuning en Embeddings preentrenados

Si usas TF-IDF, Word2Vec, GloVe, etc., puedes:

* Usar los embeddings ya entrenados.
* Entrenar un nuevo modelo de clasificación sobre ellos → eso es un tipo de transfer + fine-tuning.
* En NLP moderno con transformers (como BERT):

Usas un modelo preentrenado de Hugging Face.
* Le añades una capa de clasificación.
* Lo reentrenas con tu dataset (esto es fine-tuning puro).

### Conclusión

| Tipo de modelo               | ¿Se puede hacer fine-tuning? | ¿Cómo se aplica? |
|------------------------------|-------------------------------|------------------|
| **CNN / RNN / Transformers** | ✅ Sí                          | Congelando capas preentrenadas, añadiendo nuevas capas de salida, reentrenando con learning rate bajo |
| **XGBoost / LightGBM**       | ✅ Sí                          | Continuando el entrenamiento con nuevos datos usando `xgb_model` |
| **Regresión / SVM**          | ⚠️ Limitado                   | Ajustando hiperparámetros, reutilizando coeficientes como punto de partida (si es posible) |
| **Modelos NLP clásicos**     | ✅ Sí                          | Usando embeddings preentrenados (TF-IDF, Word2Vec, GloVe) como entrada para nuevos clasificadores |
| **Transformers (BERT, GPT)** | ✅ Sí (muy común)              | Añadiendo cabeza de clasificación y reentrenando el modelo completo o parcialmente |
